# Aprendizado Supervisionado

Importando dependências

In [1]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyRegressor
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, accuracy_score

---
## Funções Auxiliares

### Função para carregar os DataFrames
Dado o ano, mês, número de meses anteriores e o número de agrupamentos, retorna uma tupla de *DataFrames* de treino e teste

In [2]:
def load_dataframe(year, month, n_months, n_clusters):
    dirname = 'df_clusters'
    df_train = pd.read_pickle('{0}/{1}_{2}_{3}_{4}_train.pkl'.format(dirname, year, month, n_months, n_clusters))
    df_test = pd.read_pickle('{0}/{1}_{2}_{3}_{4}_test.pkl'.format(dirname, year, month, n_months, n_clusters))
    return df_train, df_test

**Exemplo:** 3 meses anteriores a Fevereiro de 2019 com 2000 agrupamentos retorna uma tupla com:
* *DataFrame* de treino com os dados dos meses de Novembro de 2018, Dezembro de 2018 e Janeiro de 2019
* *DataFrame* de teste com os dados do mês de Fevereiro de 2019

In [3]:
df_train, df_test = load_dataframe(2019, 2, 3, 2000)

In [4]:
df_train

,DATAOCORRENCIA,HORAOCORRENCIA,LATITUDE,LONGITUDE,MES,GRUPO
1,31/10/2018,21:30,-23.003776,-47.106153,1,728
3,31/10/2018,20:30,-24.042856,-46.507023,1,1445
4,31/10/2018,21:00,-23.609402,-46.929926,1,968
5,31/10/2018,23:00,-23.576057,-46.554417,1,1432
6,31/10/2018,23:00,-23.630611,-46.471524,1,923
...,...,...,...,...,...,...
22556,31/01/2019,22:20,-23.614637,-46.470736,3,384
22557,31/01/2019,22:00,-23.586411,-46.676220,3,467
22559,31/01/2019,20:05,-23.587221,-46.406488,3,1960
22560,31/01/2019,20:30,-23.452332,-46.601216,3,109


In [5]:
df_test

,DATAOCORRENCIA,HORAOCORRENCIA,LATITUDE,LONGITUDE,MES,GRUPO
0,31/01/2019,19:00,-23.510144,-46.397749,4,836
1,31/01/2019,21:45,-23.689874,-46.800888,4,410
2,31/01/2019,16:20,-23.114819,-46.586959,4,1430
4,28/01/2019,04:30,-23.497685,-46.393311,4,1043
5,31/01/2019,19:30,-23.524828,-46.630724,4,1937
...,...,...,...,...,...,...
20807,28/02/2019,20:45,-23.542405,-46.419923,4,617
20808,28/02/2019,19:00,-23.628584,-46.691692,4,1415
20809,28/02/2019,16:00,-23.595742,-46.740399,4,1312
20810,28/02/2019,22:40,-23.597856,-46.307167,4,540


### Função para transformar os atributos do DataFrame
Dado o *DataFrame* e o número de agrupamentos, a função cria uma série temporal com os números de eventos mensais de cada grupo e retorna uma tupla com os dados dos atributos e valores-alvo.

In [6]:
def get_X_y(df, n_clusters):
    
    # cria uma série temporal com o número de eventos mensais de cada grupo
    df = df.groupby(['GRUPO', 'MES']).size().reset_index()
    df.columns = ['GRUPO', 'MES', 'COUNT']
    
    # transforma o número de cada grupo em variáveis categóricas
    pipeline = ColumnTransformer([
        ('grupo', OneHotEncoder(categories=[np.arange(n_clusters, dtype='int32')]), ['GRUPO']),
        ('mes', 'passthrough', ['MES'])
    ])
    X = pipeline.fit_transform(df)
    y = df['COUNT']
    return X, y

**Exemplo:** Transformando os atributos dos *DataFrames* de treino e teste

In [7]:
X_train, y_train = get_X_y(df_train, 2000)
X_test, y_test = get_X_y(df_test, 2000)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((5899, 2001), (5899,), (1949, 2001), (1949,))

---
## Funções para Métricas de Desempenho

### Função para obter a Acurácia e a Medida F<sub>1</sub> do Modelo de Regressão
Dado um modelo de regressão, ano, mês, número de meses anteriores e número de agrupamentos, retorna uma tupla com Acurácia e a Medida F<sub>1</sub> do modelo.

In [8]:
def regressor_scores(regressor, year, month, n_month, n_cluster):
    
    # carregamento dos dataframes de treino e teste
    df_train, df_test = load_dataframe(year, month, n_month, n_cluster)

    # transformação dos atributos
    X_train, y_train = get_X_y(df_train, n_cluster)
    X_test, y_test = get_X_y(df_test, n_cluster)

    # treinamento e predição do modelo de regressão
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X_test)

    # categorização do resultado da predição e teste, baseado na mediana
    threshold = y_train.median()
    h_pred = y_pred >= threshold
    h_test = y_test >= threshold

    # retorna a Acurácia e F1 score
    return accuracy_score(h_test, h_pred), f1_score(h_test, h_pred, average='weighted')

**Exemplo:** Obtendo a Acurácia e a Medida F<sub>1</sub> do modelo de Regressão Linear com os dados de 3 meses anteriores a Fevereiro de 2019, com 2000 agrupamentos

In [9]:
lr = LinearRegression()
regressor_scores(lr, 2019, 2, 3, 2000)

(0.7131862493586455, 0.7129858199822888)

### Função para obter a Acurácia e a Medida F<sub>1</sub> do Algoritmo de Classificação
Dado um modelo de classificação, ano, mês, número de meses anteriores e número de agrupamentos, retorna uma tupla com Acurácia e a Medida F<sub>1</sub> do modelo.

In [10]:
def classifier_scores(classifier, year, month, n_month, n_cluster):
    # carregamento dos dataframes de treino e teste
    df_train, df_test = load_dataframe(year, month, n_month, n_cluster)

    # transformação dos atributos
    X_train, y_train = get_X_y(df_train, n_cluster)
    X_test, y_test = get_X_y(df_test, n_cluster)
    
    # categorização das variáveis-alvo baseado na mediana
    threshold = y_train.median()
    h_train = y_train >= threshold
    h_test = y_test >= threshold

    # treinamento e predição do modelo de classificação
    classifier.fit(X_train, h_train)
    h_pred = classifier.predict(X_test)
    
    # retorna Acurácia e a Medida F1
    return accuracy_score(h_test, h_pred), f1_score(h_test, h_pred, average='weighted')

**Exemplo:** Obtendo a Acurácia e a Medida F<sub>1</sub> do modelo de Regressão Logística com os dados de 3 meses anteriores a Fevereiro de 2019, com 2000 agrupamentos

In [11]:
lrc = LogisticRegression()
classifier_scores(lrc, 2019, 2, 3, 2000)

(0.7085684966649564, 0.7092506758606463)

---
## Testes

### Função para guardar os dados de desempenho
Dado a função de métrica de desempenho, modelo e o nome do diretório, realiza a bateria de testes para o ano de 2019 e guarda os resultados no diretório definido.

In [12]:
def save_scores(scores, model, dirname):
    
    # criação da pasta para guardar os resultados
    if not os.path.exists(dirname):
        os.makedirs(dirname)

    # ano, meses, número de meses anteriores e número de agrupamentos
    year = 2019
    months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    n_months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    n_clusters = [500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000]
        
    # estruturas de dados para as métricas de desempenho
    accuracy_scores = np.zeros((len(months), len(n_clusters), len(n_months)))
    f1_scores = np.zeros((len(months), len(n_clusters), len(n_months)))
    elapsed_time = np.zeros((len(months), len(n_clusters), len(n_months)))
    
    # para cada mês, número de agrupamentos e número de meses anteriores:
    for x, month in enumerate(months):
        for y, n_cluster in enumerate(n_clusters):
            for z, n_month in enumerate(n_months):
                start = time.time()
                # atribuímos as medidas de desempenho à estrutura de dados
                accuracy_scores[x][y][z], f1_scores[x][y][z] = scores(model, year, month, n_month, n_cluster)
                stop = time.time()
                elapsed_time[x][y][z] = stop - start
                
    # por fim, salvamos os resultados atribuídos na estrutura de dados
    np.save('{0}/{1}_accuracy'.format(dirname, year), accuracy_scores)
    np.save('{0}/{1}_f1'.format(dirname, year), f1_scores)
    np.save('{0}/{1}_time'.format(dirname, year), elapsed_time)

### Testes dos Regressores

Modelo Base (somente a média)

In [13]:
dr = DummyRegressor(strategy='mean')
save_scores(regressor_scores, dr, 'score_clusters_dr')

Regressão Linear

In [14]:
lr = LinearRegression(n_jobs=-1)
save_scores(regressor_scores, lr, 'score_clusters_lr')

Regressão com *Random Forest*

In [15]:
rfr = RandomForestRegressor(n_estimators=3, n_jobs=-1)
save_scores(regressor_scores, rfr, 'score_clusters_rfr')

Regressão com *Perceptron* de Múltiplas Camadas

In [16]:
mlpr = MLPRegressor(hidden_layer_sizes=(3,), early_stopping=True, max_iter=100000)
save_scores(regressor_scores, mlpr, 'score_clusters_mlpr')

### Testes dos Classificadores

Modelo Base (classe de maior frequência)

In [17]:
dc = DummyClassifier(strategy='prior')
save_scores(classifier_scores, dc, 'score_clusters_dc')

Regressão Logística

In [18]:
lrc = LogisticRegression(n_jobs=-1)
save_scores(classifier_scores, lrc, 'score_clusters_lrc')

Classificador com *Random Forest*

In [19]:
rfc = RandomForestClassifier(n_estimators=3, n_jobs=-1)
save_scores(classifier_scores, rfc, 'score_clusters_rfc')

Classificador com *Perceptron* de Múltiplas Camadas

In [20]:
mlpc = MLPClassifier(hidden_layer_sizes=(3,), early_stopping=True, max_iter=100000)
save_scores(classifier_scores, mlpc, 'score_clusters_mlpc')